In [174]:
import mido
import sys
import binascii
import pprint
from collections import OrderedDict
from mido import Message, MidiFile, MidiTrack

In [175]:
class TempoTrack:
    def __init__(self, track):
        self.track = track
        self.segs = {}
        self._initialize()
        
    def _initialize(self):
        segs = {}
        for i,seg in enumerate(self.track.split('ff')):
            if i==0:
                segs['header'] = seg
            else:
                seg_ = 'ff'+seg
                if seg_[0:6] != 'ff5103':
                    segs[f'meta-event-{i}'] = (seg_)
                    continue
                if len(seg_) > 12:
                    segs[f'temp-event-{i}'] = {
                        'event' : seg_[0:6],
                        'value' : seg_[6:12]
                    }
                    segs[f'delta-time-{i}'] = seg_[12:]
        self.segs = segs
    
    def print_segs(self):
        print(self.segs)
        
    def hexstring(self):
        hexstring = ''
        for k,v in self.segs.items():
            if type(v) == str:
                hexstring += v
            else:
                for kk,vv in v.items():
                    hexstring += vv
        return hexstring
        
    def div_tempo_value(self, divider):
        assert divider != 0
        for k,v in self.segs.items():
            if type(v) == str:
                continue
            assert v['event'] == 'ff5103'
            value_int = int(v['value'], 16)
            value_int_divided = int(value_int/divider)
            value_hex_divided = hex(value_int_divided)[2:]
            value_hex_divided = value_hex_divided.rjust(6, '0')
            v['value'] = value_hex_divided
            
def felt_array_into_str (arr):
    string = ""
    i = 0
    for val in arr:
        string += f'    assert [z+{i}] = {val}\n'
        i += 1
    string += f'    let z_len = {len(arr)}\n'
    return string

def chunk_into_program_string (hexstring):
    '''
    last element of the felt array is an integer indicating the hex-length of the last felt value
    '''
    arr = []
    s = hexstring
    while( len(s)>62 ):
        felt_hex = s[0:62]
        arr.append( int(felt_hex, 16) )
        s = s[62:]
    felt_hex = s
    last_length = len(felt_hex)
    arr.append( int(felt_hex, 16) )
    arr.append(last_length)
    
    string = felt_array_into_str(arr)
    return string

def tempo_track_into_program_string (tempo_track):
    string = ""
    d = tempo_track.segs
    i = 0
    for k,v in d.items():
        if type(v)==str:
            s = v
            while( len(s)>62 ):
                felt_hex = s[0:62]
                #arr.append( int(felt_hex, 16) )
                string += f'    assert [z+{i}] = {int(felt_hex, 16)}\n'
                string += f'    assert [z+{i+1}] = 62\n'
                s = s[62:]
                i += 2
            felt_hex = s
            last_length = len(felt_hex)
            string += f'    assert [z+{i}] = {int(felt_hex, 16)}\n'
            string += f'    assert [z+{i+1}] = {last_length}\n'
            i += 2
#             integer = int(v, 16)
#             length = len(v)
#             string += f'    assert [z+{i}] = {integer}\n'
#             string += f'    assert [z+{i+1}] = {length}\n'
#             i += 2
        else:
            event_integer = int(v['event'], 16)
            event_length  = len(v['event'])
            value_integer = int(v['value'], 16)
            value_length  = len(v['value'])
            assert value_length == 6
            string += f'\n    # Set Tempo at adjusted value\n'
            string += f'    assert [z+{i}] = {event_integer}\n'
            string += f'    assert [z+{i+1}] = {event_length}\n'
            string += f'    tempvar value_ = {value_integer} * tempo_multiplier\n'
            string += f'    let (adjusted_value, _) = unsigned_div_rem(value_, tempo_divider)\n'
            string += f'    assert [z+{i+2}] = adjusted_value\n'
            string += f'    assert [z+{i+3}] = 6\n' # the hex-length of tempo value is always 6
            i += 4
    string += f'    let z_len = {i}'
    return string

class CairoProgram:
    def __init__(self, name):
        program_1 = []
        program_1.append('@view')
        program_1.append(f"func {name} {{")
        program_1.append('        range_check_ptr')
        program_1.append('    } (')
        program_1.append('\n')
        program_1 = '\n'.join(program_1)
        
        program_2 = []
        program_2.append('\n')
        program_2.append('    ) -> (')
        program_2.append('        z_len : felt,')
        program_2.append('        z : felt*')
        program_2.append('    ):')
        program_2.append('    alloc_locals')
        program_2.append('')
        program_2.append('    let (local z) = alloc()')
        program_2.append('')
        program_2 = '\n'.join(program_2)
        
        program_3 = []
        program_3.append('')
        program_3.append('    return (z_len, z)')
        program_3.append('end')
        program_3 = '\n'.join(program_3)
        
        self.program_1 = program_1
        self.program_2 = program_2
        self.program_3 = program_3
        self.args = ""
        self.content = ""
    
    def populate_args (self, args):
        self.args = args
        
    def populate_content (self, content):
        self.content = content
    
    def export(self):
        '''
        export program as string
        '''
        self.program = self.program_1 + self.args + self.program_2 + self.content + self.program_3

        return self.program

### Read midi

In [202]:
MIDI_FILE = './debussy.mid'

with open(MIDI_FILE, 'rb') as f:
    hexdata = binascii.hexlify(f.read())
hexdata_str = hexdata.decode("utf-8")

MTrk = '4d54726b'
tracks = []
for i,chunk in enumerate(hexdata_str.split(MTrk)):
    if i == 0:
        header = chunk
    else:
        tracks.append (MTrk + chunk)
tempo_track = tracks[0]
music_tracks =  tracks[1:]

### Convert tracks to cairo programs

In [196]:
header_content = chunk_into_program_string(header)
header_program = CairoProgram('header')
header_program.populate_content(header_content)
header_program_str = header_program.export()

In [197]:
# tempo_content = chunk_into_program_string(tempo_track)
# tempo_program = CairoProgram('tempo')
# tempo_program.populate_content(tempo_content)
# tempo_program_str = tempo_program.export()

tempo_track_obj = TempoTrack(tempo_track)
tempo_content = tempo_track_into_program_string(tempo_track_obj)
tempo_program = CairoProgram('tempo')
tempo_program.populate_content(tempo_content)
tempo_program.populate_args('        tempo_multiplier : felt,\n        tempo_divider : felt\n')
tempo_program_str = tempo_program.export()

In [198]:
music_program_str_s = []
for i,music_track in enumerate(music_tracks):
    music_content = chunk_into_program_string(music_track)
    music_program = CairoProgram(f'music_{i}')
    music_program.populate_content(music_content)
    music_program_str = music_program.export()
    music_program_str_s.append(music_program_str)
    #music_program_str_s.append( chunk_into_program_string(music_track) )

### Export cairo contract

In [199]:
CONTRACT_TEMPLATE = ""
CONTRACT_TEMPLATE += '%lang starknet\n'
CONTRACT_TEMPLATE += '%builtins pedersen range_check'
CONTRACT_TEMPLATE += '\n'
CONTRACT_TEMPLATE += 'from starkware.cairo.common.cairo_builtins import HashBuiltin\n'
CONTRACT_TEMPLATE += 'from starkware.cairo.common.alloc import alloc\n'
CONTRACT_TEMPLATE += 'from starkware.cairo.common.math import (unsigned_div_rem)\n'

In [200]:
contract_str = CONTRACT_TEMPLATE + '\n\n' + header_program_str + '\n\n' + tempo_program_str + '\n\n'
contract_str += '\n\n'.join(music_program_str_s)

In [201]:
FILENAME = "ddddd.cairo"

with open(f'{FILENAME}', 'w') as f:
    f.write(contract_str)